# 003 Minimal example of a rolling hash

This notebook uses the low-level phylogenetic library [biomcmc-lib](https://github.com/quadram-institute-bioscience/biomcmc-lib) (commit [5975331](https://github.com/quadram-institute-bioscience/biomcmc-lib/commit/5975331ef88d1c4ec9aef9599fb6348905d289c7)).

The rolling hash (or expoential hash) is a technique to slide through a sequence while updating its hash value using only information from the current hash and the next value. One of the most popular algorithms is the Rabin-Karp algorithm. Below I start with the imnplementation in [linclust/MMseqs2](http://dx.doi.org/10.1038/s41467-018-04964-5)

In DNA context, each base is represented by a random number. In MMseqs they work with a reduced amino acid alphabet of 13 groups  

## Prototype

Here I describe a prototype (future capability of the library), not implemented yet. I am testing the rolling hash function but also shorter types `uint8_t` etc. The idea is that each of the testing functionalities will be incorporated into `biomcmc-lib`.

Below is a working version that follows closely `minclust`.

In [2]:
//%cflags:-lm
//%cflags: -I/usr/users/QIB_fr005/deolivl/Academic/Quadram/009.supersptree/biomcmc-lib/lib
//%cflags: -I/usr/users/QIB_fr005/deolivl/Academic/Quadram/009.supersptree/build/biomcmc-lib/lib
//%cflags: /usr/users/QIB_fr005/deolivl/Academic/Quadram/009.supersptree/build/biomcmc-lib/lib/.libs/libbiomcmc.a
#include <biomcmc.h>

uint16_t RAND[21] = {0x4567, 0x23c6, 0x9869, 0x4873, 0xdc51, 0x5cff, 0x944a, 0x58ec,
                           0x1f29, 0x7ccd, 0x58ba, 0xd7ab, 0x41f2, 0x1efb, 0xa9e3, 0xe146,
                           0x007c, 0x62c2, 0x0854, 0x27f8, 0x231b};// 16 bit random numbers

#define RoL(val, numbits) (val << numbits) ^ (val >> (32 - numbits))

// Transform each letter x[i] to a fixed random number RAND[x[i]] to ensure instantaneous mixing into the 16 bits
// Do XOR with RAND[x[i]] and 5-bit rotate left for each i from 1 to k
uint32_t circ_hash (const uint8_t * x, uint8_t length, const uint8_t rol){
    uint16_t h = 0x0;
    h = h ^ RAND[x[0]];                  // XOR h and ki
    for (uint8_t i = 1; i < length; ++i){
        h = RoL(h, rol);
        h ^= RAND[x[i]];                   // XOR h and ki
    }
    return h;
}

// Rolling hash for CRC hash variant: Computes hash value for next key x[0:length-1] from previous value
// hash( x[-1:length-2] ) and x_first = x[-1]
uint32_t circ_hash_next (const uint8_t * x, uint8_t length, uint8_t x_first, uint16_t h, const uint8_t rol){
    // undo INITIAL_VALUE and first letter x[0] of old key
    h ^= RoL(RAND[x_first], (5*(length-1)) % 16);// circularly permute all letters x[1:length-1] to 5 positions to left
    // h ^= RoL(RAND[x_first], (5*(length-1)) & 15); // since x%y = x & (y-1)
    h =  RoL(h, rol);// add new, last letter of new key x[1:length]
    h ^= RAND[x[length-1]];
    return h;
}

int main (){
    uint8_t i, j;
    uint8_t my_seq1[] = {1,2,3,1,2, 3,1,2,3,1, 1,1,1,1,1, 1,2,1,2,3, 1,2,1,2,1, 2,1,2}; // n=28
    uint8_t my_seq2[] = {1,2,3,4,5, 2,3,4,5,2, 3,4,5,1,1, 4,5,2,5,2, 3,4,3,2,3, 2,3,2}; 
    uint16_t my_h1 = circ_hash((const uint8_t*) my_seq1, 3, 5);
    uint16_t my_h2 = circ_hash((const uint8_t *) my_seq2, 3, 7);
    for (j = 0; j < 25; j++) {
        printf ("%3d >>  ", j);
        for (i=j;i<j+3;i++) printf ("%d ", my_seq1[i]); 
        printf (" \t %8u  \t >>  ",my_h1); // show current kmer and hash
        for (i=j;i<j+3;i++) printf ("%d ", my_seq2[i]); 
        printf (" \t %8u\n",my_h2); // show current kmer and hash
        my_h1 = circ_hash_next ((const uint8_t *) my_seq1 + j + 1, 3, my_seq1[j], my_h1, 5);
        my_h2 = circ_hash_next ((const uint8_t *) my_seq2 + j + 1, 3, my_seq2[j], my_h2, 7);
    }
}

  0 >>  1 2 3  	    23891  	 >>  1 2 3  	    64755
  1 >>  2 3 1  	    35238  	 >>  2 3 4  	    42449
  2 >>  3 1 2  	    11433  	 >>  3 4 5  	    46207
  3 >>  1 2 3  	    23891  	 >>  4 5 2  	    42985
  4 >>  2 3 1  	    35238  	 >>  5 2 3  	    48371
  5 >>  3 1 2  	    11433  	 >>  2 3 4  	    42449
  6 >>  1 2 3  	    23891  	 >>  3 4 5  	    46207
  7 >>  2 3 1  	    35238  	 >>  4 5 2  	    42985
  8 >>  3 1 1  	    38662  	 >>  5 2 3  	    48371
  9 >>  1 1 1  	    17158  	 >>  2 3 4  	    42449
 10 >>  1 1 1  	    17158  	 >>  3 4 5  	    46207
 11 >>  1 1 1  	    17158  	 >>  4 5 1  	     7238
 12 >>  1 1 1  	    17158  	 >>  5 1 1  	      198
 13 >>  1 1 1  	    17158  	 >>  1 1 4  	    48977
 14 >>  1 1 2  	    63657  	 >>  1 4 5  	    62591
 15 >>  1 2 1  	    14054  	 >>  4 5 2  	    42985
 16 >>  2 1 2  	    17577  	 >>  5 2 5  	    43135
 17 >>  1 2 3  	    23891  	 >>  2 5 2  	    42985
 18 >>  2 3 1  	    35238  	 >>  5 2 3  	    48371
 19 >>  3 1 2  	    11433  	 >>

The code above assumes a 16-bits hash, cast to 32 bits (due to the `RoL` macro). Below we use all 32 bits of hash, and also use the fact that for non-negative values, `x % y == x & (y-1)` which may be faster.

In [3]:
//%cflags:-lm
//%cflags: -I/usr/users/QIB_fr005/deolivl/Academic/Quadram/009.supersptree/biomcmc-lib/lib
//%cflags: -I/usr/users/QIB_fr005/deolivl/Academic/Quadram/009.supersptree/build/biomcmc-lib/lib
//%cflags: /usr/users/QIB_fr005/deolivl/Academic/Quadram/009.supersptree/build/biomcmc-lib/lib/.libs/libbiomcmc.a
#include <biomcmc.h>

uint32_t RAND[21] = {0x4567, 0x23c6, 0x9869, 0x4873, 0xdc51, 0x5cff, 0x944a, 0x58ec,
                           0x1f29, 0x7ccd, 0x58ba, 0xd7ab, 0x41f2, 0x1efb, 0xa9e3, 0xe146,
                           0x007c, 0x62c2, 0x0854, 0x27f8, 0x231b};

#define RoL(val, numbits) (val << numbits) ^ (val >> (32 - numbits))
uint32_t circ_hash (const uint8_t * x, uint8_t length, const uint8_t rol){
    uint32_t h = 0x0;
    h = h ^ RAND[x[0]];                  // XOR h and ki
    for (uint8_t i = 1; i < length; ++i){
        h = RoL(h, rol);
        h ^= RAND[x[i]];                 // XOR h and ki
    }
    return h;
}

uint32_t circ_hash_next (const uint8_t * x, uint8_t length, uint8_t x_first, uint32_t h, const uint8_t rol){
    uint8_t remain = (rol * (length-1)) & 31U; // since x % y = x & (y-1)
    // undo INITIAL_VALUE and first letter x[0] of old key
    //h ^= RoL(RAND[x_first], (rol * (length-1)) % 32);// circularly permute all letters x[1:length-1] to 5 positions to left
    h ^= RoL(RAND[x_first], remain);
    h =  RoL(h, rol);// add new, last letter of new key x[1:length]
    h ^= RAND[x[length-1]];
    return h;
}

int main (){
    uint8_t i, j;
    uint8_t my_seq1[] = {1,2,3,1,2, 3,1,2,3,1, 1,1,1,1,1, 1,2,1,2,3, 1,2,1,2,1, 2,1,2}; // n=28
    uint8_t my_seq2[] = {1,2,3,4,5, 2,3,4,5,2, 3,4,5,1,1, 4,5,2,5,2, 3,4,3,2,3, 2,3,2}; 
    uint32_t my_h1 = circ_hash((const uint8_t *) my_seq1, 3, 5);
    uint32_t my_h2 = circ_hash((const uint8_t *) my_seq2, 3, 7);
    for (j = 0; j < 25; j++) {
        printf ("%3d >>  ", j);
        for (i=j;i<j+3;i++) printf ("%d ", my_seq1[i]); 
        printf (" %8u  \t >>  ",my_h1); // show current kmer and hash
        for (i=j;i<j+3;i++) printf ("%d ", my_seq2[i]); 
        printf ("\t%8u\n",my_h2); // show current kmer and hash
        my_h1 = circ_hash_next ((const uint8_t *) my_seq1 + j + 1, 3, my_seq1[j], my_h1, 5);
        my_h2 = circ_hash_next ((const uint8_t *) my_seq2 + j + 1, 3, my_seq2[j], my_h2, 7);
    }
}

  0 >>  1 2 3  10247507  	 >>  1 2 3 	146668787
  1 >>  2 3 1  40405414  	 >>  2 3 4 	641639889
  2 >>  3 1 2  19213481  	 >>  3 4 5 	309507199
  3 >>  1 2 3  10247507  	 >>  4 5 2 	926590953
  4 >>  2 3 1  40405414  	 >>  5 2 3 	393460979
  5 >>  3 1 2  19213481  	 >>  2 3 4 	641639889
  6 >>  1 2 3  10247507  	 >>  3 4 5 	309507199
  7 >>  2 3 1  40405414  	 >>  4 5 2 	926590953
  8 >>  3 1 1  19240710  	 >>  5 2 3 	393460979
  9 >>  1 1 1   9126662  	 >>  2 3 4 	641639889
 10 >>  1 1 1   9126662  	 >>  3 4 5 	309507199
 11 >>  1 1 1   9126662  	 >>  4 5 1 	926555206
 12 >>  1 1 1   9126662  	 >>  5 1 1 	388890822
 13 >>  1 1 1   9126662  	 >>  1 1 4 	148946769
 14 >>  1 1 2   9173161  	 >>  1 4 5 	144700543
 15 >>  1 2 1  10237670  	 >>  4 5 2 	926590953
 16 >>  2 1 2  40191145  	 >>  5 2 5 	393455743
 17 >>  1 2 3  10247507  	 >>  2 5 2 	640985065
 18 >>  2 3 1  40405414  	 >>  5 2 3 	393460979
 19 >>  3 1 2  19213481  	 >>  2 3 4 	641639889
 20 >>  1 2 1  10237670  	 >>  3 4 3 	30

## transforming DNA bases into hash values

Above, we have a single hash value `RAND[]` for each base, but we can create a stream of values.

1. we define a translation table from DNA to $0...3$ for non-ambiguous data (or some other encoding). Notice that we must know its reverse complement.
2. hash values inspired by `RAND[]` are then created. They are "salted" with large primes.

The version currently implemented in `biomcmc-lib` is, for non-ambiguous DNA:
```c
uint8_t dna_in_2_bits[256][2] = {{0xff}};

void
initialize_dna_to_bit_tables (void) {
    int i;
    for (i = 0; i < 256; i++) dna_in_2_bits[i][0] = dna_in_2_bits[i][1] = 4; // calling function must check if < 4
    dna_in_2_bits['A'][0] = 0;   dna_in_2_bits['A'][1] = 3;  /*  A  <-> T  */
    dna_in_2_bits['C'][0] = 1;   dna_in_2_bits['C'][1] = 2;  /*  C  <-> G  */
    dna_in_2_bits['G'][0] = 2;   dna_in_2_bits['G'][1] = 1;  /*  G  <-> C  */
    dna_in_2_bits['T'][0] = 3;   dna_in_2_bits['T'][1] = 0;  /*  T  <-> A  */
    dna_in_2_bits['U'][0] = 3;   dna_in_2_bits['U'][1] = 0;  /*  U  <-> A  */
}
```
It just returns a small number, that is concatenated to form the k-mer. I.e. a sequence `ACGGT` becomes an integer with binary `00 01 10 10 01`. 

Below is the tentative code:

In [1]:
//%cflags:-lm
//%cflags: -I/usr/users/QIB_fr005/deolivl/Academic/Quadram/009.supersptree/biomcmc-lib/lib
//%cflags: -I/usr/users/QIB_fr005/deolivl/Academic/Quadram/009.supersptree/build/biomcmc-lib/lib
//%cflags: /usr/users/QIB_fr005/deolivl/Academic/Quadram/009.supersptree/build/biomcmc-lib/lib/.libs/libbiomcmc.a
#include <biomcmc.h>
const char dna[] = 
    "AAACCACCTCCTCAATATATCCTAGATGTCATCATCTAAATACTCTAAATGAGATAACAG \
    GTTCTTAACATTCTCATAGGTTTCTCGTACAGCCACTCGTTTACCATCACTGGTGAAGAT \
    ACAGGTATTACCTTCGTCAACTGTAATGAATGTAATGTTATTGATATTAAACTCCACTGG \
    GTTATATAATGGTTGACTGTTTTGTTGATAACATATAGCCCCCTGTATTAGAGGGCTAGG \
    TTCATTGTTCTGTGTGGGGTTAGTCTCTTCTACATCAGCAGAGGTCTCTTCCTCGTATGC \
    TTCATACGGTACGTGTTCAGTAACAACAATCAAATTAGGAAACACTACAAGTTGTTCATC";

uint32_t rand_hash_list[] = {0x4567, 0x23c6, 0x9869, 0x4873, 0xdc51}; // size = 5
uint32_t prime_salt_list[] = { // size = 64 
  0x343EAF9F, 0x75BD32B,  0x5E1C5A87, 0x343EFDAF, 0x1FDCDBEF, 0x6389CB,   0x1FDCE507, 0x1FDCE3C7, 
  0x1FDCE15F, 0x75BD431,  0x34C8B0F,  0x5397FEF,  0x87413,    0x6389C9,   0x34C8B23,  0x343EFDB5, 
  0x75BD479,  0x5398013,  0x5397FCD,  0x87407,    0x75BD413,  0x343EAF4B, 0x343EB047, 0x6389CF, 
  0x5E1C5A2F, 0x343EFD31, 0x34C8B59,  0x343EFD57, 0x1FDCDC19, 0x5E1C5B0D, 0x34C8B5D,  0x343EB035, 
  0x75BD307,  0x343EAF7B, 0x343EAEF5, 0x75BD433,  0x1FDCDBD5, 0x1FDCE3B7, 0x343EFD19, 0x5398057, 
  0x34C8B51,  0x638971,   0x1FDCE16F, 0x63896F,   0x343EB04D, 0x873FD,    0x63896B,   0x1FDCDC9B, 
  0x5E1C5B1F, 0x343EFCEB, 0x5398007,  0x1FDCE51F, 0x63897B,   0x5E1C5B17, 0x5E1C5ABD, 0xFF85, 
  0x1FDCDC25, 0x873EB,    0x5E1C5AB1, 0x75BD30D,  0x1FDCDCC1, 0xFF8B,     0x5397FC1,  0x5398001};

uint32_t**
new_dna_salted_hash_encoding (uint8_t salt) {
    uint8_t i, j;
    uint32_t **shash = (uint32_t**) biomcmc_malloc (2 * sizeof (uint32_t*)); // opposite order as dna_in_2_bits[]
    for (i = 0; i < 2;i++) shash[i] = (uint32_t*) biomcmc_malloc (256 * sizeof (uint32_t));
    
    for (i = 0; i < 256; i++) shash[0][i] = shash[1][i] = 4; // anything else treated as fifth state
    shash[0]['A'] = shash[0]['a'] = 0; shash[1]['A'] = shash[1]['a'] = 3;  /*  A  <-> T  */
    shash[0]['C'] = shash[0]['c'] = 1; shash[1]['C'] = shash[1]['c'] = 2;  /*  C  <-> G  */
    shash[0]['G'] = shash[0]['g'] = 2; shash[1]['G'] = shash[1]['g'] = 1;  /*  G  <-> C  */
    shash[0]['T'] = shash[0]['t'] = 3; shash[1]['T'] = shash[1]['t'] = 0;  /*  T  <-> A  */
    shash[0]['U'] = shash[0]['u'] = 3; shash[1]['U'] = shash[1]['u'] = 0;  /*  U  <-> A  */
    
    salt &= 63; // modulus, we only have 64 salts
    /** now we transform the indexes for their equiv. hash values; all have same salt */
    for (j = 0; j < 2; ++j) for (i = 0; i < 256; ++i) shash[j][i] = rand_hash_list[shash[j][i]] + prime_salt_list[salt];
    return shash;
}

void
del_dna_salted_hash_encoding (uint32_t** shash) {
    if (!shash) return;
    if (shash[1]) free (shash[1]);
    if (shash[0]) free (shash[0]);
    free (shash);
}


#define RoL(val, numbits) ((val) << (numbits)) ^ ((val) >> (32 - (numbits)))
// FIXME: code below incomplete 
uint32_t circular_hash_start (const char* dna, 
                              uint8_t kmer_length, 
                              const uint8_t rol, 
                              const uint32_t* shashcode, // just one of the two vectors
                              const bool homopol_compress) {
    uint32_t h = 0x0; 
    h = h ^ shashcode[(uint8_t)dna[0]];                  // XOR h and ki
    for (uint8_t i = 1; i < kmer_length; ++i){
        h = RoL(h, rol);
        h ^= RAND[x[i]];                 // XOR h and ki
    }
    return h;
}

uint32_t circular_hash_next (const uint8_t * x, uint8_t length, uint8_t x_first, uint32_t h, const uint8_t rol){
    uint8_t remain = (rol * (length-1)) & 31U; // since x % y = x & (y-1)
    // undo INITIAL_VALUE and first letter x[0] of old key
    //h ^= RoL(RAND[x_first], (rol * (length-1)) % 32);// circularly permute all letters x[1:length-1] to 5 positions to left
    h ^= RoL(RAND[x_first], remain);
    h =  RoL(h, rol);// add new, last letter of new key x[1:length]
    h ^= RAND[x[length-1]];
    return h;
}



/tmp/tmpwprnos3t.c: In function ‘circular_hash_start’:
/tmp/tmpwprnos3t.c:64:14: error: ‘RAND’ undeclared (first use in this function); did you mean ‘NAN’?
         h ^= RAND[x[i]];                 // XOR h and ki
              ^~~~
              NAN
/tmp/tmpwprnos3t.c:64:14: note: each undeclared identifier is reported only once for each function it appears in
/tmp/tmpwprnos3t.c:64:19: error: ‘x’ undeclared (first use in this function)
         h ^= RAND[x[i]];                 // XOR h and ki
                   ^
/tmp/tmpwprnos3t.c: In function ‘circular_hash_next’:
/tmp/tmpwprnos3t.c:73:14: error: ‘RAND’ undeclared (first use in this function); did you mean ‘NAN’?
     h ^= RoL(RAND[x_first], remain);
              ^
/tmp/tmpwprnos3t.c:53:29: note: in definition of macro ‘RoL’
 #define RoL(val, numbits) ((val) << (numbits)) ^ ((val) >> (32 - (numbits)))
                             ^~~
[C kernel] GCC exited with code 1, the executable will not be executed

## conclusions

`linclust` may have a "bug" since a five is hardcoded instead of `rol` in ` RoL(RAND[x_first], (5*(length-1)) % 16);`